Capstone

In [2]:
from bs4 import BeautifulSoup
import pandas as pd # library to process data as dataframes
import requests

Extract Table

In [3]:
with open('abcd.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [4]:
table = soup.find_all('table')[0]
table_rows = table.find_all('tr')

for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]

In [5]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df =pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighborhood"])
df=df[df.Borough!="Not assigned"]
df.drop(df.index[0],inplace=True)
df.reset_index(drop=True,inplace=True)
df['Neighborhood']=df['Neighborhood'].astype(str).str[:-2].astype(str)

Display New Table

Group together by PostalCode and keep Neighborhoods

In [6]:
df=df.groupby('PostalCode').agg({'Borough':'first', 
                             'Neighborhood': ', '.join})

In [7]:
df.shape

(103, 2)

In [8]:
import pandas as pd2

In [9]:
df2 = pd2.read_csv('Geospatial_Coordinates.csv')

In [10]:
df2.rename(columns={'Postal Code':'PostalCode'}, 
                 inplace=True)
df2.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
import pandas as pd3
df3=pd3.merge(df, df2, on="PostalCode")
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Roug, Malver",43.806686,-79.194353
1,M1C,Scarborough,"Highland Cree, Rouge Hil, Port Unio",43.784535,-79.160497
2,M1E,Scarborough,"Guildwoo, Morningsid, West Hil",43.763573,-79.188711
3,M1G,Scarborough,Wobur,43.770992,-79.216917
4,M1H,Scarborough,Cedarbra,43.773136,-79.239476


Extract Borough with Toronto

In [28]:
Toronto =  df3[df3['Borough'].str.contains("Toronto")]
Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beache,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth Wes, Riverdal",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches Wes, India Bazaa",43.668999,-79.315572
43,M4M,East Toronto,Studio Distric,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Par,43.728020,-79.388790


Import necessary libraries

In [30]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.3.0               |        py36_1001         533 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.7 MB

The following NEW packages will be INSTALLED:

    altair:  2.3.0-py36_1001   conda-forge
    branca:  0.3.1-py_0        conda-forge
    folium:  0.5.0-py_0        conda-forge
    vincent: 0.4.

Four Square Details

In [31]:
CLIENT_ID = 'MQDXQRPPQBJZO5ZFBEG0IOIQVGXJTO2HSH10ZPPV0ER5SDUE' # your Foursquare ID
CLIENT_SECRET = 'KXX55YU5EBO3XNLKLQ05OKOU2R152EONQ1SK2XCR5HOBIRHZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MQDXQRPPQBJZO5ZFBEG0IOIQVGXJTO2HSH10ZPPV0ER5SDUE
CLIENT_SECRET:KXX55YU5EBO3XNLKLQ05OKOU2R152EONQ1SK2XCR5HOBIRHZ


In [32]:
df3.loc[0, 'Neighborhood']

'Roug, Malver'

In [33]:
neighborhood_latitude = df3.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df3.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df3.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Roug, Malver are 43.806686299999996, -79.19435340000001.


In [34]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=MQDXQRPPQBJZO5ZFBEG0IOIQVGXJTO2HSH10ZPPV0ER5SDUE&client_secret=KXX55YU5EBO3XNLKLQ05OKOU2R152EONQ1SK2XCR5HOBIRHZ&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c35706e9fb6b714c675b7ba'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [38]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
Toronto_venues = getNearbyVenues(names=Toronto['Neighborhood'],
                                   latitudes=Toronto['Latitude'],
                                   longitudes=Toronto['Longitude']
                                  )

The Beache
The Danforth Wes, Riverdal
The Beaches Wes, India Bazaa
Studio Distric
Lawrence Par
Davisville Nort
North Toronto Wes
Davisvill
Moore Par, Summerhill Eas
Deer Par, Forest Hill S, Rathnell, South Hil, Summerhill Wes
Rosedal
Cabbagetow, St. James Tow
Church and Wellesle
Harbourfron, Regent Par
Ryerso, Garden Distric
St. James Tow
Berczy Par
Central Bay Stree
Adelaid, Kin, Richmon
Harbourfront Eas, Toronto Island, Union Statio
Design Exchang, Toronto Dominion Centr
Commerce Cour, Victoria Hote
Roselaw
Forest Hill Nort, Forest Hill Wes
The Anne, North Midtow, Yorkvill
Harbor, University of Toront
Chinatow, Grange Par, Kensington Marke
CN Towe, Bathurst Qua, Island airpor, Harbourfront Wes, King and Spadin, Railway Land, South Niagar
Stn A PO Boxes 25 The Esplanad
First Canadian Plac, Underground cit
Christi
Dovercourt Villag, Dufferi
Little Portuga, Trinit
Brockto, Exhibition Plac, Parkdale Villag
High Par, The Junction Sout
Parkdal, Roncesvalle
Runnymed, Swanse
Business Reply M

In [42]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1712, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beache,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beache,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beache,43.676357,-79.293031,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,The Beache,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth Wes, Riverdal",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [44]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()

#### Let's print each neighborhood along with the top 5 most common venues

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaid, Kin, Richmon",Coffee Shop,Café,Thai Restaurant,American Restaurant,Steakhouse,Gym,Restaurant,Hotel,Asian Restaurant,Bar
1,Berczy Par,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Italian Restaurant,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market,Pub
2,"Brockto, Exhibition Plac, Parkdale Villag",Breakfast Spot,Café,Coffee Shop,Pet Store,Climbing Gym,Convenience Store,Burrito Place,Bar,Stadium,Caribbean Restaurant
3,Business Reply Mail Processing Centre 969 Easter,Yoga Studio,Auto Workshop,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Park,Comic Shop,Butcher
4,"CN Towe, Bathurst Qua, Island airpor, Harbourf...",Airport Lounge,Airport Terminal,Airport Service,Plane,Sculpture Garden,Boutique,Boat or Ferry,Harbor / Marina,Airport Gate,Airport
5,"Cabbagetow, St. James Tow",Restaurant,Coffee Shop,Pizza Place,Café,Italian Restaurant,Bakery,Park,Pub,Breakfast Spot,Convenience Store
6,Central Bay Stree,Coffee Shop,Café,Italian Restaurant,Chinese Restaurant,Bar,Burger Joint,Thai Restaurant,Salad Place,Bubble Tea Shop,Spa
7,"Chinatow, Grange Par, Kensington Marke",Bar,Café,Vegetarian / Vegan Restaurant,Bakery,Dumpling Restaurant,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,Mexican Restaurant,Gaming Cafe
8,Christi,Grocery Store,Café,Park,Convenience Store,Nightclub,Baby Store,Restaurant,Athletics & Sports,Diner,Italian Restaurant
9,Church and Wellesle,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Fast Food Restaurant,Café,Men's Store,Burrito Place


In [51]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [53]:
Toronto_merged = Toronto

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beache,43.676357,-79.293031,0,Coffee Shop,Pub,Music Venue,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
41,M4K,East Toronto,"The Danforth Wes, Riverdal",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Brewery,Bubble Tea Shop,Restaurant,Pub
42,M4L,East Toronto,"The Beaches Wes, India Bazaa",43.668999,-79.315572,0,Park,Sandwich Place,Movie Theater,Liquor Store,Fast Food Restaurant,Burger Joint,Fish & Chips Shop,Burrito Place,Food & Drink Shop,Steakhouse
43,M4M,East Toronto,Studio Distric,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Middle Eastern Restaurant,Juice Bar,Fish Market,Latin American Restaurant
44,M4N,Central Toronto,Lawrence Par,43.728020,-79.388790,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Women's Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [55]:
# create map
latitude = 43.676357
longitude = -79.293031
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters